# make dataset

#### I got the studylevel dataframe by [step2 make_dataframe](https://www.kaggle.com/kunihikofurugori/siim-step1-get-imginfo).

#### Next step, I will make the dataset about study level.

## my public notebook 

### [step1 get_imageinformation](https://www.kaggle.com/kunihikofurugori/siim-step1-get-imginfo).

### [step2 make_dataframe](https://www.kaggle.com/kunihikofurugori/siim-step1-get-imginfo).

### [step3-1 renew-imglev_ds](https://www.kaggle.com/kunihikofurugori/siim-step3-1-renew-imglev-ds)

### [step3-2 renew-studylev_ds](https://www.kaggle.com/kunihikofurugori/siim-step3-1-renew-studylev-ds)

## study level strategy (pre augment image)

### 1. exclude side black part
### 2. adjust image histgram

In [ ]:
!pip install python-gdcm
import numpy as np 
import pandas as pd 
import cv2,random
import pydicom
import gdcm
from skimage.exposure import exposure, equalize_hist,equalize_adapthist
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
import glob
import os
from PIL import Image, ImageStat
import albumentations as A

In [ ]:
os.mkdir("image1024")
os.mkdir("augimage1024")
#os.mkdir("augcomparison")

In [ ]:
df = pd.read_csv("../input/step2-make-dataframe/train_study.csv")

df["histflag"] = 0
index = df[(df["mean"]< 90) & (df["var"]<40)].index
print(len(index))
df.loc[index,"histflag"] = 1
print(len(df))
targetmean, targetstd = df["mean"].values.mean(),df["var"].values.mean()
#df = df.sort_values("norm_var",ascending=False)
df['fold'] = 0
df.fold = df.fold.apply(lambda x: random.randint(0,3))
df

In [ ]:
targetmean, targetstd

In [ ]:
def get_transform(image_size):
    return A.Compose(
            [

            A.CLAHE(clip_limit=(1,3),p=0.5),
            A.RandomBrightnessContrast(
                    brightness_limit=0.1, 
                    contrast_limit=0.1,
                    p=1.0),
            A.RandomGamma(gamma_limit = (90, 110),p=1.0)
            ],p=1.0)

            
def optimization_imagehist(row,image, image_crop, trytimes=100):
    best_loss = targetmean**2 + targetstd**2
    for _ in range(trytimes):
        if image_crop:
            if random.uniform(0,1)<0.5:
                xmin, xmax = int(row.crop_xmin), int(row.crop_xmax)
                ymin, ymax = int(row.crop_ymin), int(row.crop_ymax)
                images = image[xmin:xmax,ymin:ymax]
            else:
                images = image
        transform = get_transform((image.shape[0],image.shape[1]))
        sample = transform(image=image)["image"]
        calc_loss = (sample.mean() - targetmean)**2 + (sample.std() - targetstd)**2
        if calc_loss < best_loss:
            best_loss = calc_loss
            outimg = sample
    return outimg

In [ ]:
def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

def resize(array, size1,size2, keep_ratio=True, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size1, size2), resample)
        im = expand2square(im, background_color=0)
    else:
        im = im.resize((size1, size2), resample)
    
    return im

In [ ]:
dfs = []
dfs.append(df[df.fold ==0])
dfs.append(df[df.fold ==1])
dfs.append(df[df.fold ==2])
dfs.append(df[df.fold ==3])
print(len(dfs[0]),len(dfs[1]),len(dfs[2]),len(dfs[3]))
print(len(df))

In [ ]:
def run(fold):
    for idx,(index,row) in enumerate(dfs[fold].iterrows()):
        image = read_xray(row.path)
        
        

        im = resize(image, size1=1024, size2 =1024, keep_ratio=False)
        im.save("image1024" + "/" + row.id + ".png")
        if row.histflag == 1:
            image = optimization_imagehist(row,image, image_crop=False)
        
        if row.augflag == 1:
            xmin, xmax = int(row.crop_xmin), int(row.crop_xmax)
            ymin, ymax = int(row.crop_ymin), int(row.crop_ymax)
            augimage = image[xmin:xmax,ymin:ymax]
            
            im = resize(augimage, size1=1024, size2 =1024, keep_ratio=False)
            im.save("augimage1024" + "/" + row.id + ".png")
            
        if row.augflag == 0:
            augimage = optimization_imagehist(row,image, image_crop=False)
            im = resize(augimage, size1=1024, size2 =1024, keep_ratio=False)
            im.save("augimage1024" + "/" + row.id + ".png")
         
        #if (image.mean() != augimage.mean()) | (image.std() != augimage.std()):
            #fig, ax= plt.subplots(1,2)
            #ax[0].imshow(image)
            #ax[0].axis('off')
            #ax[0].set_title("before")
            #ax[1].imshow(augimage)
            #ax[1].axis('off')
            #plt.title(f"img_path:{row.id}", x=0, y=1.3, size=15)
            #plt.show()
            #fig.savefig("augcomparison/" + row.id +".png",dpi=80)
            #plt.close()
            #del fig, ax
            
        

        if idx%500==0:
            print(idx)

In [ ]:
#run(0)

In [ ]:
from joblib import Parallel, delayed
Parallel(n_jobs=4, backend="threading")(delayed(run)(i) for i in range(4))

In [ ]:
!zip -qr image1024.zip image1024
!rm -r image1024
!zip -qr augimage1024.zip augimage1024
!rm -r augimage1024
#!zip -qr augcomparison.zip augcomparison
#!rm -r augcomparison



In [ ]:
df.to_csv("train.csv")